# **1. Introduction** <a class="anchor" id="1"></a>
## **1.2. Import packages**

In [1]:
# Remember: library imports are ALWAYS at the top of the script, no exceptions!
import math
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from math import ceil
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from pandas_profiling import ProfileReport
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
# Feature selection
from sklearn.feature_selection import VarianceThreshold

from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances

from sklearn import random_projection

from scipy.cluster.hierarchy import dendrogram

## Plotting Functions

In [2]:
def boxplot_func(df):
    plot_features = df.columns
    #Prepare figure layout
    rows = int(math.ceil(df.shape[1]/8))
    sns.set()
    fig, axes = plt.subplots(nrows = rows, ncols=8, figsize=(16,4*rows))
  
    # Draw the boxplots
    for i in zip(axes.flatten(), plot_features):
        sns.boxplot(data=df.loc[:,i[1]], ax=i[0])      
        i[0].set_title(i[1])
        i[0].set_ylabel("")
        for tick in i[0].get_xticklabels():
            tick.set_rotation(-25)
    
    # Finalize the plot
    plt.subplots_adjust(wspace=2,hspace = 0.5)
    fig.suptitle("Box plots", fontsize=25)
    sns.despine(bottom=True)
    plt.show()
    
def histplot_func(df):
    plot_features = df.columns
    #Prepare figure layout
    rows = int(math.ceil(df.shape[1]/8))
    sns.set()
    fig, axes = plt.subplots(nrows = rows, ncols=8, figsize=(24,2*rows))
       
    # Draw the boxplots
    for i in zip(axes.flatten(), plot_features):
        sns.histplot(data=df.loc[:,i[1]], bins=20,ax=i[0])
        i[0].set_xlabel(i[1])
        i[0].set_ylabel("")
      
    # Finalize the plot
    plt.subplots_adjust(wspace=0.5,hspace = 0.5)
    fig.suptitle("Histograms", fontsize=15)
    sns.despine(bottom=True)
    plt.show()
    
def corrheatmap(dataframe):
    sns.set(style="white")

    # Compute the correlation matrix
    corr = dataframe.corr() #Getting correlation of numerical variables

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool) #Return an array of zeros (Falses) with the same shape and type as a given array
    mask[np.triu_indices_from(mask)] = True #The upper-triangle array is now composed by True values

    # Set up the matplotlib figure
    fig, ax = plt.subplots(figsize=(16, 8))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True) #Make a diverging palette between two HUSL colors. Return a matplotlib colormap object.

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, center=0, square=True, annot=True,annot_kws={"fontsize":10}, linewidths=.5,vmin = -1, vmax = 1, ax=ax)

    # Layout
    plt.subplots_adjust(top=0.95)
    plt.suptitle("Correlation matrix", fontsize=15)
    plt.yticks(rotation=0)
    # Fixing the bug of partially cut-off bottom and top cells
    b, t = plt.ylim() # discover the values for bottom and top
    b += 0.5 # Add 0.5 to the bottom
    t -= 0.5 # Subtract 0.5 from the top
    plt.ylim(b, t) # update the ylim(bottom, top) values

    plt.show()

## **1.3. Import data**

In [3]:
data = pd.read_csv('data/donors.csv',index_col='CONTROLN', low_memory=False)
print("The data set has total %d records and %d features" %data.shape)

The data set has total 95412 records and 475 features


# **2. Pre-processing data:** <a class="anchor" id="1"></a>

After conducting **Exploratory Data Analysis (Appendix 1)** for 475 features of the original data set, we have listed out the most significant features that we consider to be important for the clustering task.  
Thus, from now, we will continue with these selected features:

In [4]:
# Import the database we create for selecting the features:
features_list = pd.read_csv('Selected_Features.csv',index_col=0)
features_list.head()

,FeaturesName,Description,Category,SubCategory,Selected
Index,,,,,
1,ODATEDW,Origin Date. Date of donor's first gift,Characteristics,NaN,x
2,OSOURCE,Origin Source,Characteristics,NaN,x
3,TCODE,Donor title code,Characteristics,NaN,x
4,STATE,State abbreviation (a nominal/symbolic field),Characteristics,NaN,x
5,ZIP,Zipcode (a nominal/symbolic field),Characteristics,NaN,x


In [5]:
# Get the selected features
selected_feats = features_list[features_list['Selected'] == 'x']['FeaturesName']
df = data[selected_feats].copy()
df.dtypes

# function to update slected features dataframe everytime we adjust the data
def update_selected_feats_list(df, features_list):
    selected_feats = features_list.loc[features_list['FeaturesName'].isin(df.columns),:]
    return selected_feats

## **Feature engineering:** <a class="anchor" id="1"></a>


### **2.1. Getting the Age of every donor:** <a class="anchor" id="1"></a>
We notice that there is one feature called DOB - Day of Birth of the donors, so we thought it would we useful to generate the age of the donors as one feature. The donor's age will be measured at the time the donor received the most recent promotion which is recorded in 'MAXADATE' feature.

In [6]:
df['DOB'].head()

CONTROLN
95515     1957-12-01
148535    1972-02-01
15078            NaN
172556    1948-01-01
7112      1940-01-01
Name: DOB, dtype: object

In [7]:
#converting the DOB column to datetime type
parsed_dates = pd.to_datetime(df.DOB, format='%Y-%m-%d' , errors='coerce') #errors='coerce' meant to return NaN in case of exception
df['DOB'] = parsed_dates
df['MAXADATE'] = pd.to_datetime(df['MAXADATE'], format='%Y-%m-%d' , errors='coerce')
# Getting the age of every donor today
df['AGE'] = ((df['MAXADATE'] - df['DOB']).dt.days/365).floordiv(other=1).astype('int',errors='ignore')

# Append new features to the feature list
feats = pd.DataFrame([['AGE', "Age of the donor","Characteristics","","x" ]],
                    columns=features_list.columns)
features_list= features_list.append(feats,ignore_index=True)

# Now, drop de DOB column as it is no longer useful
df.drop('DOB',inplace=True,axis = 1)

In [8]:
# Check for the box plot and distribution
df.boxplot(column=['AGE'])
# Here we can see that we have outliers, we will access this in the up coming session

<AxesSubplot:>

### **2.2. Converting binary features of mailing activity:** <a class="anchor" id="1"></a>
There are features containing binary information but they are not in the right data type for processing, so we convert them to binary type

In [9]:
df.loc[:, 'COLLECT1':'PLATES'].head()

,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES
CONTROLN,,,,,,,,,,,,,,,,,,
95515,,,,,,,,,,,,,,,,,,
148535,,,,,,,,,,,,,,,,,,
15078,,,,,,,,,,,,,,,,,,
172556,,,,,,,,,,,,,,,,,,
7112,,,Y,Y,,,,Y,,,Y,,Y,,Y,,Y,


In [10]:
#Replace the values for booleans
df.loc[:, 'COLLECT1':'PLATES'] = df.loc[:, 'COLLECT1':'PLATES'].replace(' ',0)
df.loc[:, 'COLLECT1':'PLATES'] = df.loc[:, 'COLLECT1':'PLATES'].replace('Y',1)
df.loc[:, 'COLLECT1':'PLATES'].head()

,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES
CONTROLN,,,,,,,,,,,,,,,,,,
95515,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
148535,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15078,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
172556,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7112,0,0,1,1,0,0,0,1,0,0,1,0,1,0,1,0,1,0


In [11]:
# Plot the result
plotdf = df.loc[:, 'COLLECT1':'PLATES'].apply(pd.value_counts)
#plot pie charts for all the interests
sns.set()
fig, axes = plt.subplots(3, ceil(len(plotdf.columns.to_list()) / 3), figsize=(20,11))
for ax, feat in zip(axes.flatten(), plotdf.columns.to_list()):
    ax.pie(plotdf[feat])
    ax.set_title(feat, y=-0.13)
    ax.legend(plotdf.index)
plt.show() 

### **2.3. Generate RFA status of the each donor before they are considered to be Lapsed Donor:** <a class="anchor" id="1"></a>
Because all the donors which is investigated in this dataset is all Lapsed Donors as the date of the lasted promotion, so we will not able to undestand the 'Recency' charateristics if we only consider the most recent code of the donor (all 'Lapsed').  

Therefore, we thought it would be interesting if we get the most recent type of code of each donor before they are considered to become Lapsed Donors

We will consider the data in features **Donor's RFA status as of XXXX promotion date (XXXX is from 'RFA_3'to 'RFA_24')**

In [12]:
# Replace the blank values with NaN value
df.loc[:,'RFA_3':'RFA_24'] = df.loc[:,'RFA_3':'RFA_24'].replace({' ' : np.NaN})
# Find the second most recent type of a donor by the left-most available values from RFA_3 to RFA_24
df['RFA_USEDTO'] = df.loc[:,'RFA_3':'RFA_24'].fillna(method = 'bfill',axis = 1)['RFA_3']
# Renerate R, F, A features
df['RFA_USEDTO_R'] = df['RFA_USEDTO'].apply(lambda x: str(x)[0])
df['RFA_USEDTO_F'] = df['RFA_USEDTO'].apply(lambda x: str(x)[1])
df['RFA_USEDTO_A'] = df['RFA_USEDTO'].apply(lambda x: str(x)[2])

# Add to the selected features list
feats = pd.DataFrame([['RFA_USEDTO_R', "Recency code of the previous status of the donor","Promotion and Gift","","x" ],
                    ['RFA_USEDTO_F', "Frequency code of the previous status of the donor","Promotion and Gift","","x"], 
                    ['RFA_USEDTO_A', "Amount code of the previous status of the donor","Promotion and Gift","","x"]],
                    columns=features_list.columns)
features_list= features_list.append(feats,ignore_index=True)

# Now drop the features that is no longer useful
df.drop(labels='RFA_USEDTO',axis=1,inplace = True, errors='ignore')
# Check for the result
df.loc[:,'RFA_USEDTO_R':'RFA_USEDTO_A'].head()

,RFA_USEDTO_R,RFA_USEDTO_F,RFA_USEDTO_A
CONTROLN,,,
95515,S,4,E
148535,A,2,G
15078,S,4,E
172556,S,4,E
7112,A,2,F


In [13]:
# Drop the RFA status of donor as they are no longer useful
df.drop(labels=df.loc[:,'RFA_3':'RFA_24'].columns,axis=1,inplace=True,errors='ignore')
df.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,MDMAUD,DOMAIN,HOMEOWNR,CHILD03,...,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,AGE,RFA_USEDTO_R,RFA_USEDTO_F,RFA_USEDTO_A
CONTROLN,,,,,,,,,,,,,,,,,,,,,
95515,2009-01-01,GRI,0,IL,61081,,XXXX,T2,,,...,4,E,X,X,X,C,59.0,S,4,E
148535,2014-01-01,BOA,1,CA,91326,,XXXX,S1,H,,...,2,G,X,X,X,A,45.0,A,2,G
15078,2010-01-01,AMH,1,NC,27017,,XXXX,R2,U,,...,4,E,X,X,X,C,NaN,S,4,E
172556,2007-01-01,BRY,0,CA,95953,,XXXX,R2,U,,...,4,E,X,X,X,C,69.0,S,4,E
7112,2006-01-01,,0,FL,33176,,XXXX,S2,H,,...,2,F,X,X,X,A,77.0,A,2,F


In [14]:
selected_feats = features_list.loc[features_list['FeaturesName'].isin(df.columns),:]
selected_feats.head()

,FeaturesName,Description,Category,SubCategory,Selected
0,ODATEDW,Origin Date. Date of donor's first gift,Characteristics,NaN,x
1,OSOURCE,Origin Source,Characteristics,NaN,x
2,TCODE,Donor title code,Characteristics,NaN,x
3,STATE,State abbreviation (a nominal/symbolic field),Characteristics,NaN,x
4,ZIP,Zipcode (a nominal/symbolic field),Characteristics,NaN,x


In [15]:
# Summary by category
pd.DataFrame(selected_feats.groupby('Category')['FeaturesName'].count())

,FeaturesName
Category,
Characteristics,23
Interests,19
Mail response,14
Neighborhood,75
Promotion and Gift,27


# **3. Data cleanning:** <a class="anchor" id="1"></a>

In [16]:
df

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,MDMAUD,DOMAIN,HOMEOWNR,CHILD03,...,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,AGE,RFA_USEDTO_R,RFA_USEDTO_F,RFA_USEDTO_A
CONTROLN,,,,,,,,,,,,,,,,,,,,,
95515,2009-01-01,GRI,0,IL,61081,,XXXX,T2,,,...,4,E,X,X,X,C,59.0,S,4,E
148535,2014-01-01,BOA,1,CA,91326,,XXXX,S1,H,,...,2,G,X,X,X,A,45.0,A,2,G
15078,2010-01-01,AMH,1,NC,27017,,XXXX,R2,U,,...,4,E,X,X,X,C,NaN,S,4,E
172556,2007-01-01,BRY,0,CA,95953,,XXXX,R2,U,,...,4,E,X,X,X,C,69.0,S,4,E
7112,2006-01-01,,0,FL,33176,,XXXX,S2,H,,...,2,F,X,X,X,A,77.0,A,2,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184568,2016-01-01,ASE,1,AK,99504,,XXXX,C2,,,...,1,G,X,X,X,C,NaN,F,1,G
122706,2016-01-01,DCD,1,TX,77379,,XXXX,C1,H,,...,1,F,X,X,X,A,47.0,P,1,F
189641,2015-01-01,MBC,1,MI,48910,,XXXX,C3,,,...,3,E,X,X,X,B,59.0,S,4,E


## 3.1. Check for features interpretability:
In this section, we will access the ability to interpret the feature based on its description, number of unique values and nulls values.  
We also try to engineer some features for better interpretation

### 3.1.1. Characteristics features:

In [17]:
selected_feats[selected_feats['Category'] == 'Characteristics']

,FeaturesName,Description,Category,SubCategory,Selected
0,ODATEDW,Origin Date. Date of donor's first gift,Characteristics,NaN,x
1,OSOURCE,Origin Source,Characteristics,NaN,x
2,TCODE,Donor title code,Characteristics,NaN,x
3,STATE,State abbreviation (a nominal/symbolic field),Characteristics,NaN,x
4,ZIP,Zipcode (a nominal/symbolic field),Characteristics,NaN,x
5,MAILCODE,Mail Code,Characteristics,NaN,x
13,MDMAUD,The Major Donor Matrix code,Characteristics,NaN,x
14,DOMAIN,DOMAIN/Cluster code. A nominal or symbolic field.,Characteristics,NaN,x
15,HOMEOWNR,Home Owner Flag,Characteristics,NaN,x
16,CHILD03,Presence of Children age 0-3,Characteristics,NaN,x


In [18]:
char_feats = selected_feats[selected_feats['Category'] == 'Characteristics']['FeaturesName']

In [19]:
df[char_feats]

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,MDMAUD,DOMAIN,HOMEOWNR,CHILD03,...,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MAJOR,WEALTH2,GEOCODE,GEOCODE2,AGE
CONTROLN,,,,,,,,,,,,,,,,,,,,,
95515,2009-01-01,GRI,0,IL,61081,,XXXX,T2,,,...,NaN,NaN,F,NaN,0,,5.0,,C,59.0
148535,2014-01-01,BOA,1,CA,91326,,XXXX,S1,H,,...,1.0,6.0,M,9.0,16,,9.0,02,A,45.0
15078,2010-01-01,AMH,1,NC,27017,,XXXX,R2,U,,...,NaN,3.0,M,1.0,2,,1.0,,C,NaN
172556,2007-01-01,BRY,0,CA,95953,,XXXX,R2,U,,...,NaN,1.0,F,4.0,2,,0.0,,C,69.0
7112,2006-01-01,,0,FL,33176,,XXXX,S2,H,,...,1.0,3.0,F,2.0,60,,NaN,,A,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184568,2016-01-01,ASE,1,AK,99504,,XXXX,C2,,,...,NaN,NaN,M,NaN,0,,NaN,,C,NaN
122706,2016-01-01,DCD,1,TX,77379,,XXXX,C1,H,,...,1.0,7.0,M,9.0,1,,NaN,,A,47.0
189641,2015-01-01,MBC,1,MI,48910,,XXXX,C3,,,...,NaN,NaN,M,NaN,0,,NaN,,B,59.0


**a) Feature engineering:**

There are some columns contain missing values but in different type of format rather than NaN format. Thus, in this section, we will replace them with numpy.NaN values for better processing:

In [20]:
df[char_feats].isin([' ']).sum(axis = 0)

ODATEDW         0
OSOURCE       928
TCODE           0
STATE           0
ZIP             0
MAILCODE    94013
MDMAUD          0
DOMAIN       2316
HOMEOWNR    22228
CHILD03     94266
CHILD07     93846
CHILD12     93601
CHILD18     92565
NUMCHLD         0
INCOME          0
GENDER       2957
WEALTH1         0
HIT             0
MAJOR       95118
WEALTH2         0
GEOCODE     80168
GEOCODE2      187
AGE             0
dtype: int64

In [21]:
# Except for MAILCODE where ' ' is actually a value, we will replace all other ' ' values as NaN
for col in df[char_feats].isin([' ']).sum(axis = 0).index:
    if col == 'MAILCODE':
        next
    else:
        df[col] = df[col].replace(' ', np.NaN)

In [22]:
#checking all the values for the Gender
#M = Male
#F = Female
#U = Unknown
#J = Joint Account
print(df['GENDER'].unique())

#replacing unknown values to 'U'
df['GENDER'].replace(' ','U',inplace=True)
df['GENDER'].replace('A','U',inplace=True)
df['GENDER'].replace('C','U',inplace=True)

#checking the difference
print(df['GENDER'].unique())

['F' 'M' nan 'C' 'U' 'J' 'A']
['F' 'M' nan 'U' 'J']


**b) Check for unique values:**

In [23]:
df[char_feats].describe(include=['O'])

,ODATEDW,OSOURCE,STATE,ZIP,MAILCODE,MDMAUD,DOMAIN,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,GENDER,MAJOR,GEOCODE,GEOCODE2
count,95412,94484,95412,95412,95412,95412,93096,73184,1146,1566,1811,2847,92455,294,15244,95093
unique,54,895,57,19938,2,28,16,2,3,3,3,3,4,1,7,4
top,2015-01-01,MBC,CA,85351,,XXXX,R2,H,M,M,M,M,F,X,12,A
freq,15358,4539,17343,61,94013,95118,13623,52354,869,1061,1149,1442,51277,294,3914,34484


In [24]:
def unique_check(df, threshold):
    table = df.describe(include=['O'])
    highly_unique = []
    for col in table.columns:
        f = table.loc['freq',col]/table.loc['count',col]
        if f > threshold:
            highly_unique.append(col)
            print(col, ', percentage of top unique value = %.2f %%' %(100*f))
    if len(highly_unique) == 0:
        print('There is no column has too high percentage of a value')

In [25]:
unique_check(df[char_feats], 0.9)

MAILCODE , percentage of top unique value = 98.53 %
MDMAUD , percentage of top unique value = 99.69 %
MAJOR , percentage of top unique value = 100.00 %


In [26]:
#columns are not interesting and have lots of null values
df.drop(['MAILCODE','MDMAUD','CHILD03','CHILD07','CHILD12','CHILD18','MAJOR'],inplace=True,axis = 1)
# Update the seleted features list
selected_feats = update_selected_feats_list(df, features_list)
char_feats = selected_feats[selected_feats['Category'] == 'Characteristics']['FeaturesName']

**c) Check for correlation**

In [27]:
corrheatmap(df[char_feats])

In [28]:
#find features with correlations greater than 0.9 in order to remove them
def correlation_check(df, limit = 0.90, drop = False):
    corr = df.corr()
    mask = np.triu(np.ones(corr.shape), k=1).astype(bool)
    corr_no_diag = corr.where(mask)
    coll = [c for c in corr_no_diag.columns if any(abs(corr_no_diag[c]) > limit)]
    if drop is True:        
        df.drop(coll,axis = 1,inplace=True)
        print('Dropped:', coll)
        return df
    else:
        print('Columns having high correlation:', coll) 

In [29]:
correlation_check(df[char_feats])

Columns having high correlation: []


**d) Check for Nulls values:**

In [30]:
df[char_feats].isna().sum()

ODATEDW         0
OSOURCE       928
TCODE           0
STATE           0
ZIP             0
DOMAIN       2316
HOMEOWNR    22228
NUMCHLD     83026
INCOME      21286
GENDER       2957
WEALTH1     44732
HIT             0
WEALTH2     43823
GEOCODE     80168
GEOCODE2      319
AGE         23883
dtype: int64

Too many null values for NUMCHLD and GEOCODE so we will drop these columns

In [31]:
#too many null values for NUMCHLD and GEOCODE so we will drop these columns
df.drop(['NUMCHLD','GEOCODE'],inplace=True,axis = 1,errors='ignore')
# Update the seleted features list
selected_feats = update_selected_feats_list(df, features_list)
char_feats = selected_feats[selected_feats['Category'] == 'Characteristics']['FeaturesName']

In [32]:
# Summary by category
pd.DataFrame(selected_feats.groupby('Category')['FeaturesName'].count())

,FeaturesName
Category,
Characteristics,14
Interests,19
Mail response,14
Neighborhood,75
Promotion and Gift,27


### 3.1.4. Promotion and Gift features:

In [33]:
PG_feats = selected_feats[selected_feats['Category'] == 'Promotion and Gift']['FeaturesName']
# selected features
selected_feats[selected_feats['Category'] == 'Promotion and Gift']

,FeaturesName,Description,Category,SubCategory,Selected
404,CARDPROM,Lifetime number of card promotions received to,Promotion and Gift,Summary,x
405,MAXADATE,Date of the most recent promotion received (in,Promotion and Gift,Summary,x
406,NUMPROM,Lifetime number of promotions received to date,Promotion and Gift,Summary,x
407,CARDPM12,Number of card promotions received in the last,Promotion and Gift,Summary,x
408,NUMPRM12,Number of promotions received in the last 12,Promotion and Gift,Summary,x
453,RAMNTALL,Dollar amount of lifetime gifts to date,Promotion and Gift,Summary,x
454,NGIFTALL,Number of lifetime gifts to date,Promotion and Gift,Summary,x
455,CARDGIFT,Number of lifetime gifts to card promotions to...,Promotion and Gift,Summary,x
456,MINRAMNT,Dollar amount of smallest gift to date,Promotion and Gift,Summary,x
457,MINRDATE,Date associated with the smallest gift to date,Promotion and Gift,Summary,x


**a) Feature engineering:**

There are 4 date-time features that we can assume that they will not have much interpretation power for clustering.  
Those features are: **'MINRDATE', 'MAXRDATE', 'NEXTDATE' and 'TIMELAG'**.  

Thus, we decided to drop them:

In [34]:
# Remove un-relevant features
df = df.drop(columns=['MINRDATE','MAXRDATE','NEXTDATE','TIMELAG'],errors='ignore')

The remaining date-time features are 'MAXADATE', 'LASTDATE' and 'FISTDATE'. These features are useful to generate two important features in RFM (Recency, Frequency and Monetary) analysis which are Recency and Frequency.  

The 2 new features to generate are:
- Time in days since the last promotion and the last gift from the donor: 'RECENCY' = 'MAXADATE' - 'LASTDATE'  
- Time in days since the last promotion and the first gift from the donor: 'LIFETIME' = 'MAXADATE' - 'FISTDATE' 

In [35]:
# convert data to datetime 
df['MAXADATE'] = pd.to_datetime(df['MAXADATE'])
df['LASTDATE'] = pd.to_datetime(df['LASTDATE'])
df['FISTDATE'] = pd.to_datetime(df['FISTDATE'])
# measure the recency and lifetime of donors
df['RECENCY'] = (df['MAXADATE'] - df['LASTDATE'])/pd.to_timedelta(1, unit='D')
df['LIFETIME'] = (df['MAXADATE'] - df['FISTDATE'])/pd.to_timedelta(1, unit='D')

# Add new features to the metadata
feats = pd.DataFrame([['RECENCY', "Time in days since the last promotion and the last gift from the donor","Promotion and Gift","","x" ],
                    ['LIFETIME', "Time in days since the last promotion and the first gift from the donor","Promotion and Gift","","x"]], 
                    columns=selected_feats.columns)
features_list= features_list.append(feats,ignore_index=True)
                      
# Then we drop the datetime columns:
df = df.drop(columns=['MAXADATE','FISTDATE','LASTDATE'],errors='ignore')

# Update the seleted features list
selected_feats = update_selected_feats_list(df, features_list)
PG_feats = selected_feats[selected_feats['Category'] == 'Promotion and Gift']['FeaturesName']

In [36]:
histplot_func(df.loc[:,['RECENCY','LIFETIME']])

**b) Check for unique values:**

In [37]:
df[PG_feats].describe(include=['O'])

,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,RFA_USEDTO_R,RFA_USEDTO_F,RFA_USEDTO_A
count,95412,95412,95412,95412,95412,95412,95412,95412
unique,1,4,5,4,5,8,4,7
top,L,F,X,X,X,A,1,F
freq,95412,46964,95118,95118,95118,60156,48358,46170


In [38]:
unique_check(df[PG_feats], 0.9)

RFA_2R , percentage of top unique value = 100.00 %
MDMAUD_R , percentage of top unique value = 99.69 %
MDMAUD_F , percentage of top unique value = 99.69 %
MDMAUD_A , percentage of top unique value = 99.69 %


We can clearly see that features **'RFA_2R', 'MDMAUD_R', 'MDMAUD_F' and 'MDMAUD_A'** having only 1 values for mostly all of their records, thus they are considered redundant in this cluster analysis

In [39]:
# Drop these redundant columns
df = df.drop(columns=['RFA_2R', 'MDMAUD_R', 'MDMAUD_F','MDMAUD_A'],errors='ignore')
# Update the seleted features list
selected_feats = update_selected_feats_list(df, features_list)
PG_feats = selected_feats[selected_feats['Category'] == 'Promotion and Gift']['FeaturesName']

**c) Check for correlation**

In [40]:
corrheatmap(df[PG_feats])

In [41]:
correlation_check(df[PG_feats])

Columns having high correlation: ['NUMPROM', 'CARDGIFT', 'LIFETIME']


In [42]:
# Drop these redundant columns
df = df.drop(columns=['NUMPROM', 'CARDGIFT', 'LIFETIME'])
# Update the seleted features list
selected_feats = update_selected_feats_list(df, features_list)
PG_feats = selected_feats[selected_feats['Category'] == 'Promotion and Gift']['FeaturesName']

**d) Check for Nulls values:**

In [43]:
df[PG_feats].isna().sum()

CARDPROM        0
CARDPM12        0
NUMPRM12        0
RAMNTALL        0
NGIFTALL        0
MINRAMNT        0
MAXRAMNT        0
LASTGIFT        0
AVGGIFT         0
RFA_2F          0
RFA_2A          0
RFA_USEDTO_R    0
RFA_USEDTO_F    0
RFA_USEDTO_A    0
RECENCY         0
dtype: int64

## 3.2. Check for outliners:

In [44]:
def IQR_OutlierRemover(df,column,n):
    #give the function the column that you want to filter and the IQR multiplicator    
    # Compute the IQR
    q1= df[column].quantile(0.25)
    q3= df[column].quantile(0.75)
    Iqr = q3 - q1

    # Compute upper and lower limit (lower_limit = Q1 -1.5*IQR | upper_limit = Q3 + 1.5*IQR)
    lower_lim = q1 - n*Iqr
    upper_lim = q3 + n*Iqr

    initial_len = df.shape[0]

    df = df[~((df[column] < lower_lim) | (df[column] > upper_lim))]

    len_afterremov = df.shape[0]

    print('Percentage of data kept after removing outliers:', np.round(len_afterremov / initial_len, 4))

    return df

In [45]:
df = IQR_OutlierRemover(df,'AGE',2)

Percentage of data kept after removing outliers: 1.0


In [46]:
df = IQR_OutlierRemover(df,'POP901',12.5)

Percentage of data kept after removing outliers: 0.9882


In [47]:
df = IQR_OutlierRemover(df,'POP90C5',5)

Percentage of data kept after removing outliers: 0.9869


In [48]:
#Removing Outlier of some columns without getting rid of lots of data
df = IQR_OutlierRemover(df,'ETH1',5)
df = IQR_OutlierRemover(df,'ETH2',13)
df = IQR_OutlierRemover(df,'ETH3',10)
df = IQR_OutlierRemover(df,'ETH4',13)
df = IQR_OutlierRemover(df,'ETH5',12.5)
df = IQR_OutlierRemover(df,'ETH12',1.5)

Percentage of data kept after removing outliers: 0.9852
Percentage of data kept after removing outliers: 0.9901
Percentage of data kept after removing outliers: 0.9908
Percentage of data kept after removing outliers: 0.9921
Percentage of data kept after removing outliers: 0.9929
Percentage of data kept after removing outliers: 0.9907


## 3.3. 